In [1]:
import numpy as np
from collections import namedtuple
import random
import torch
from torch import nn, optim
import torch.nn.functional as F

from tetris_env import Tetris

BATCH_SIZE = 32
CAPACITY = 10000

Transition = namedtuple("Transition", ("state", "action", "next_state", "reward"))

GAMMA = 0.99
MAX_STEPS = 14
NUM_EPISODES = 5000
TARGET_SCORE = 4

class ReplayMemory:

    def __init__(self, CAPACITY):
        self.capacity = CAPACITY
        self.memory = []
        self.index = 0
    
    def push(self, state, action, state_next, reward):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.index] = Transition(state, action, state_next, reward)
        self.index = (self.index + 1) % self.capacity
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

class Net(nn.Module):
    def __init__(self, n_in, n_mid, n_out):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_in, n_mid)
        self.fc2 = nn.Linear(n_mid, n_mid)
        self.fc3_adv = nn.Linear(n_mid, n_out)
        self.fc3_v = nn.Linear(n_mid, 1)

    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))

        adv = self.fc3_adv(h2)
        val = self.fc3_v(h2).expand(-1, adv.size(1))

        output = val + adv - adv.mean(1, keepdim=True).expand(-1, adv.size(1))

        return output

class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions # まぁ0..39の40個なんだけどね
        self.memory = ReplayMemory(CAPACITY)

        n_in, n_mid, n_out = num_states, 32, num_actions
        self.main_q_network = Net(n_in, n_mid, n_out)
        self.target_q_network = Net(n_in, n_mid, n_out)

        # print(self.main_q_network)

        self.optimizer = optim.Adam(self.main_q_network.parameters(), lr=0.0001)
    
    def replay(self):
        if len(self.memory) < BATCH_SIZE:
            return

        self.batch, self.state_batch, self.action_batch, self.reward_batch, self.non_final_next_states = self.make_minibatch()
        self.expected_state_action_values = self.get_expected_state_action_values()
        self.update_main_q_network()

    def decide_action(self, state, episode):
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            self.main_q_network.eval()
            with torch.no_grad():
                action = self.main_q_network(state).max(1)[1].view(1, 1)
        
        else:
            action = torch.LongTensor([[random.randrange(self.num_actions)]])
        
        return action
    
    def make_minibatch(self):
        transitions = self.memory.sample(BATCH_SIZE)
        batch = Transition(*zip(*transitions))
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])

        return batch, state_batch, action_batch, reward_batch, non_final_next_states

    def get_expected_state_action_values(self):
        self.main_q_network.eval()
        self.target_q_network.eval()

        self.state_action_values = self.main_q_network(self.state_batch).gather(1, self.action_batch)
        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None, self.batch.next_state)))

        next_state_values = torch.zeros(BATCH_SIZE)

        a_m = torch.zeros(BATCH_SIZE).type(torch.LongTensor)

        a_m[non_final_mask] = self.main_q_network(self.non_final_next_states).detach().max(1)[1]

        a_m_non_final_next_states = a_m[non_final_mask].view(-1, 1)

        next_state_values[non_final_mask] = self.target_q_network(self.non_final_next_states).gather(1, a_m_non_final_next_states).detach().squeeze()

        expected_state_action_values = self.reward_batch + GAMMA * next_state_values

        return expected_state_action_values

    def update_main_q_network(self):
        self.main_q_network.train()
        loss = F.smooth_l1_loss(self.state_action_values, self.expected_state_action_values.unsqueeze(1))

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    
    def update_target_q_network(self):
        self.target_q_network.load_state_dict(self.main_q_network.state_dict())

class Agent:
    def __init__(self, num_states, num_actions):
        self.brain = Brain(num_states, num_actions)

    def update_q_function(self):
        self.brain.replay()
    
    def get_action(self, state, episode):
        action = self.brain.decide_action(state, episode)
        return action
    
    def memorize(self, state, action, state_next, reward):
        self.brain.memory.push(state, action, state_next, reward)
    
    def update_target_q_function(self):
        self.brain.update_target_q_network()

## ここから要注意

def make_observation(am, field):
    observation = np.array(field)
    observation = np.append(observation, am)
    return observation

class Environment:
    def __init__(self):
        self.env = Tetris()
        # self.num_states = self.env.observation_space.shape[0]
        self.num_states = 211 # <= 20 * 11 + 1(降ってきているブロック)
        # self.num_actions = self.env.action_space.n
        self.num_actions = 40 # <= [north, east, south, west] * 10

        self.agent = Agent(self.num_states, self.num_actions)
    
    def run(self):
        episode_10_list = np.zeros(10)

        complete_episodes = 0 # scoreがTARGET_SCORE以上になった試行数
        episode_final = False
        # frames = []

        for episode in range(NUM_EPISODES):
            am, field = self.env.reset()
            observation = make_observation(am, field)
            state = torch.from_numpy(observation).type(torch.FloatTensor)
            state = torch.unsqueeze(state, 0)

            sum_reward = 0
            sum_density_reward = 0

            for step in range(MAX_STEPS):
                action = self.agent.get_action(state, episode)
                am, field, step_reward, done = self.env.step(action.item())
                observation_next = make_observation(am, field)

                real_score = step_reward // 1
                sum_reward += real_score
                sum_density_reward += step_reward - real_score

                if done:
                    state_next = None

                    episode_10_list = np.hstack((episode_10_list[1:], sum_reward))

                    if step < (MAX_STEPS-1): # 窒息死
                        reward = torch.FloatTensor([-10.0])
                    else:
                        reward = torch.FloatTensor([float(step_reward)]) # 報酬はいつも通り
                    
                    if sum_reward < TARGET_SCORE:
                        complete_episodes = 0
                    else:
                        complete_episodes += 1
                
                else:
                    reward = torch.FloatTensor([float(step_reward)])
                    state_next = torch.from_numpy(observation_next).type(torch.FloatTensor)
                    state_next = torch.unsqueeze(state_next, 0)
                
                self.agent.memorize(state, action, state_next, reward)
                self.agent.update_q_function()
                state = state_next
                    
                if done:
                    print(f"{episode} Episode: Finished with score {sum_reward} ; density_score {sum_density_reward / 14} : 10試行の平均SCORE = {episode_10_list.mean():.1f}")
                    if(episode % 2 == 0):
                        self.agent.update_target_q_function()
                    break
                    
            if episode_final is True:
                print("episode final's score:", sum_reward)
                break

            if complete_episodes >= 10:
                print(f"10回連続{TARGET_SCORE}点越え")
                episode_final = True

tetris_envi = Environment()
tetris_envi.run()

0 Episode: Finished with score 0.0 ; density_score 0.21212069676355386 : 10試行の平均SCORE = 0.0
1 Episode: Finished with score 0.0 ; density_score 0.2391267462696034 : 10試行の平均SCORE = 0.0
2 Episode: Finished with score 0.0 ; density_score 0.2112018140589569 : 10試行の平均SCORE = 0.0
3 Episode: Finished with score 0.0 ; density_score 0.16221492645641253 : 10試行の平均SCORE = 0.0
4 Episode: Finished with score 0.0 ; density_score 0.16925221603793036 : 10試行の平均SCORE = 0.0
5 Episode: Finished with score 0.0 ; density_score 0.3206858088591834 : 10試行の平均SCORE = 0.0
6 Episode: Finished with score 0.0 ; density_score 0.24882086167800452 : 10試行の平均SCORE = 0.0
7 Episode: Finished with score 0.0 ; density_score 0.3241950113378685 : 10試行の平均SCORE = 0.0
8 Episode: Finished with score 0.0 ; density_score 0.2753735153735154 : 10試行の平均SCORE = 0.0
9 Episode: Finished with score 0.0 ; density_score 0.214142920571492 : 10試行の平均SCORE = 0.0
10 Episode: Finished with score 0.0 ; density_score 0.26798752834467116 : 10試行の平均SCORE 

89 Episode: Finished with score 0.0 ; density_score 0.3509297052154195 : 10試行の平均SCORE = 0.0
90 Episode: Finished with score 0.0 ; density_score 0.35399659863945576 : 10試行の平均SCORE = 0.0
91 Episode: Finished with score 0.0 ; density_score 0.32691840372512637 : 10試行の平均SCORE = 0.0
92 Episode: Finished with score 0.0 ; density_score 0.3212585034013605 : 10試行の平均SCORE = 0.0
93 Episode: Finished with score 0.0 ; density_score 0.29835749291742214 : 10試行の平均SCORE = 0.0
94 Episode: Finished with score 0.0 ; density_score 0.2624555276656117 : 10試行の平均SCORE = 0.0
95 Episode: Finished with score 0.0 ; density_score 0.2924123046347725 : 10試行の平均SCORE = 0.0
96 Episode: Finished with score 0.0 ; density_score 0.2535171031975543 : 10試行の平均SCORE = 0.0
97 Episode: Finished with score 0.0 ; density_score 0.3313188002473716 : 10試行の平均SCORE = 0.0
98 Episode: Finished with score 0.0 ; density_score 0.30131249702678276 : 10試行の平均SCORE = 0.0
99 Episode: Finished with score 0.0 ; density_score 0.3831199752628324 : 10試

179 Episode: Finished with score 0.0 ; density_score 0.4027891156462586 : 10試行の平均SCORE = 0.0
180 Episode: Finished with score 0.0 ; density_score 0.30715855572998424 : 10試行の平均SCORE = 0.0
181 Episode: Finished with score 0.0 ; density_score 0.3313226059654631 : 10試行の平均SCORE = 0.0
182 Episode: Finished with score 0.0 ; density_score 0.35709853638425076 : 10試行の平均SCORE = 0.0
183 Episode: Finished with score 0.0 ; density_score 0.3186250257678829 : 10試行の平均SCORE = 0.0
184 Episode: Finished with score 0.0 ; density_score 0.36713286713286714 : 10試行の平均SCORE = 0.0
185 Episode: Finished with score 0.0 ; density_score 0.3654761904761905 : 10試行の平均SCORE = 0.0
186 Episode: Finished with score 0.0 ; density_score 0.33311965811965816 : 10試行の平均SCORE = 0.0
187 Episode: Finished with score 0.0 ; density_score 0.33662428607806755 : 10試行の平均SCORE = 0.0
188 Episode: Finished with score 0.0 ; density_score 0.3690809190809191 : 10試行の平均SCORE = 0.0
189 Episode: Finished with score 0.0 ; density_score 0.3830735930

269 Episode: Finished with score 0.0 ; density_score 0.34230333158904586 : 10試行の平均SCORE = 0.1
270 Episode: Finished with score 0.0 ; density_score 0.38249567892425035 : 10試行の平均SCORE = 0.1
271 Episode: Finished with score 0.0 ; density_score 0.38914965986394573 : 10試行の平均SCORE = 0.1
272 Episode: Finished with score 0.0 ; density_score 0.38106575963718825 : 10試行の平均SCORE = 0.1
273 Episode: Finished with score 0.0 ; density_score 0.3443498168498168 : 10試行の平均SCORE = 0.1
274 Episode: Finished with score 0.0 ; density_score 0.3145908060193775 : 10試行の平均SCORE = 0.1
275 Episode: Finished with score 0.0 ; density_score 0.39434065934065937 : 10試行の平均SCORE = 0.1
276 Episode: Finished with score 0.0 ; density_score 0.2752436429117101 : 10試行の平均SCORE = 0.0
277 Episode: Finished with score 0.0 ; density_score 0.37341269841269853 : 10試行の平均SCORE = 0.0
278 Episode: Finished with score 0.0 ; density_score 0.3669047619047619 : 10試行の平均SCORE = 0.0
279 Episode: Finished with score 0.0 ; density_score 0.354240759

359 Episode: Finished with score 0.0 ; density_score 0.3676190476190476 : 10試行の平均SCORE = 0.0
360 Episode: Finished with score 0.0 ; density_score 0.3437212787212788 : 10試行の平均SCORE = 0.0
361 Episode: Finished with score 0.0 ; density_score 0.3625414268271411 : 10試行の平均SCORE = 0.0
362 Episode: Finished with score 0.0 ; density_score 0.3472088229231086 : 10試行の平均SCORE = 0.0
363 Episode: Finished with score 0.0 ; density_score 0.37642857142857145 : 10試行の平均SCORE = 0.0
364 Episode: Finished with score 0.0 ; density_score 0.31807641238313505 : 10試行の平均SCORE = 0.0
365 Episode: Finished with score 0.0 ; density_score 0.3832419168133455 : 10試行の平均SCORE = 0.0
366 Episode: Finished with score 0.0 ; density_score 0.35642857142857143 : 10試行の平均SCORE = 0.0
367 Episode: Finished with score 0.0 ; density_score 0.31451144094001243 : 10試行の平均SCORE = 0.0
368 Episode: Finished with score 0.0 ; density_score 0.41290764790764783 : 10試行の平均SCORE = 0.0
369 Episode: Finished with score 0.0 ; density_score 0.3480437023

...

2850 Episode: Finished with score 0.0 ; density_score 0.5010204081632652 : 10試行の平均SCORE = 0.1
2851 Episode: Finished with score 0.0 ; density_score 0.5011904761904761 : 10試行の平均SCORE = 0.1
2852 Episode: Finished with score 0.0 ; density_score 0.4841156462585034 : 10試行の平均SCORE = 0.1
2853 Episode: Finished with score 0.0 ; density_score 0.48715728715728707 : 10試行の平均SCORE = 0.1
2854 Episode: Finished with score 0.0 ; density_score 0.4366048237476809 : 10試行の平均SCORE = 0.1
2855 Episode: Finished with score 1.0 ; density_score 0.4911904761904761 : 10試行の平均SCORE = 0.2
2856 Episode: Finished with score 0.0 ; density_score 0.48081632653061224 : 10試行の平均SCORE = 0.2
2857 Episode: Finished with score 0.0 ; density_score 0.46142857142857147 : 10試行の平均SCORE = 0.2
2858 Episode: Finished with score 0.0 ; density_score 0.4901587301587301 : 10試行の平均SCORE = 0.1
2859 Episode: Finished with score 0.0 ; density_score 0.487612863327149 : 10試行の平均SCORE = 0.1
2860 Episode: Finished with score 0.0 ; density_score 0.52

2938 Episode: Finished with score 0.0 ; density_score 0.5638775510204083 : 10試行の平均SCORE = 0.2
2939 Episode: Finished with score 0.0 ; density_score 0.5176190476190475 : 10試行の平均SCORE = 0.2
2940 Episode: Finished with score 0.0 ; density_score 0.5060997732426304 : 10試行の平均SCORE = 0.2
2941 Episode: Finished with score 0.0 ; density_score 0.5261904761904761 : 10試行の平均SCORE = 0.2
2942 Episode: Finished with score 0.0 ; density_score 0.4456153370439085 : 10試行の平均SCORE = 0.0
2943 Episode: Finished with score 0.0 ; density_score 0.4805442176870748 : 10試行の平均SCORE = 0.0
2944 Episode: Finished with score 0.0 ; density_score 0.48809523809523814 : 10試行の平均SCORE = 0.0
2945 Episode: Finished with score 0.0 ; density_score 0.4864285714285715 : 10試行の平均SCORE = 0.0
2946 Episode: Finished with score 0.0 ; density_score 0.5656235827664399 : 10試行の平均SCORE = 0.0
2947 Episode: Finished with score 0.0 ; density_score 0.5681632653061224 : 10試行の平均SCORE = 0.0
2948 Episode: Finished with score 0.0 ; density_score 0.561

3026 Episode: Finished with score 0.0 ; density_score 0.4857967429396 : 10試行の平均SCORE = 0.1
3027 Episode: Finished with score 1.0 ; density_score 0.5348072562358276 : 10試行の平均SCORE = 0.1
3028 Episode: Finished with score 0.0 ; density_score 0.577437641723356 : 10試行の平均SCORE = 0.1
3029 Episode: Finished with score 0.0 ; density_score 0.5003854875283446 : 10試行の平均SCORE = 0.1
3030 Episode: Finished with score 0.0 ; density_score 0.5398299319727891 : 10試行の平均SCORE = 0.1
3031 Episode: Finished with score 1.0 ; density_score 0.5312755102040815 : 10試行の平均SCORE = 0.2
3032 Episode: Finished with score 1.0 ; density_score 0.46775510204081633 : 10試行の平均SCORE = 0.3
3033 Episode: Finished with score 0.0 ; density_score 0.45081632653061227 : 10試行の平均SCORE = 0.3
3034 Episode: Finished with score 0.0 ; density_score 0.5479251700680272 : 10試行の平均SCORE = 0.3
3035 Episode: Finished with score 0.0 ; density_score 0.5415873015873016 : 10試行の平均SCORE = 0.3
3036 Episode: Finished with score 0.0 ; density_score 0.545238

3115 Episode: Finished with score 0.0 ; density_score 0.5239569160997732 : 10試行の平均SCORE = 0.0
3116 Episode: Finished with score 0.0 ; density_score 0.5188239538239537 : 10試行の平均SCORE = 0.0
3117 Episode: Finished with score 0.0 ; density_score 0.5633219954648527 : 10試行の平均SCORE = 0.0
3118 Episode: Finished with score 0.0 ; density_score 0.5090476190476191 : 10試行の平均SCORE = 0.0
3119 Episode: Finished with score 0.0 ; density_score 0.552517006802721 : 10試行の平均SCORE = 0.0
3120 Episode: Finished with score 1.0 ; density_score 0.5362868480725623 : 10試行の平均SCORE = 0.1
3121 Episode: Finished with score 0.0 ; density_score 0.5169387755102042 : 10試行の平均SCORE = 0.1
3122 Episode: Finished with score 0.0 ; density_score 0.49968079539508103 : 10試行の平均SCORE = 0.1
3123 Episode: Finished with score 0.0 ; density_score 0.5084126984126983 : 10試行の平均SCORE = 0.1
3124 Episode: Finished with score 0.0 ; density_score 0.5222902494331066 : 10試行の平均SCORE = 0.1
3125 Episode: Finished with score 0.0 ; density_score 0.5318

OSError: exception: access violation writing 0x0000000208851018